In [8]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
import mlflow
# pipeline
from sklearn.pipeline import make_pipeline

In [4]:
# os.environ["AWS_PROFILE"] = "mlops"

TRACKING_SERVER_HOST = "ec2-54-234-169-161.compute-1.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
print(f"MLFLOW TRACKING URI: {mlflow.get_tracking_uri()}")
mlflow.search_experiments()

mlflow.set_experiment("green-taxi-duration-prediction")

MLFLOW TRACKING URI: http://ec2-54-234-169-161.compute-1.amazonaws.com:5000


2025/09/05 12:25:06 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration-prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://koomi-mlflow-artifacts-remote/2', creation_time=1757075106520, experiment_id='2', last_update_time=1757075106520, lifecycle_stage='active', name='green-taxi-duration-prediction', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet'.format(year=2021, month=1))
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year:04d}-{month:02d}.parquet'.format(year=2021, month=2))

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)



In [9]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)
    
    pipeline = make_pipeline(DictVectorizer(),
                             RandomForestRegressor(**params, n_jobs=-1)
                             )


    pipeline.fit(dict_train, y_train)
    dv = pipeline.named_steps['dictvectorizer']
    y_pred = pipeline.predict(dict_val)
    rmse = root_mean_squared_error(y_pred, y_val)
    
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")
    


2025/09/05 12:27:05 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/05 12:27:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run magnificent-bird-812 at: http://ec2-54-234-169-161.compute-1.amazonaws.com:5000/#/experiments/2/runs/808c4e3ce88c4edcb3587442b13678ed
🧪 View experiment at: http://ec2-54-234-169-161.compute-1.amazonaws.com:5000/#/experiments/2
